# Calulcates the Predictions Accuracy

In [3]:
import pandas as pd

year = 2025
week = 5

In [4]:
actual = pd.read_csv(f'cfbd_{year}_{week}_games.csv')
predictions = pd.read_csv(f'season{year}_week{week}_predictions.csv')

In [5]:
actual.head()

,season,week,homeId,homeTeam,awayTeam,awayId,homePoints,awayPoints,home_year_x,home_elo,...,away_year_y,away_epa,away_epaAllowed,away_successRate,away_successRateAllowed,away_explosiveness,away_explosivenessAllowed,away_year,away_rank,away_points
0,2025,5,151,East Carolina,Army,349,28,6,2025,1538,...,2025,0.216947,0.177844,0.452975,0.458338,0.968576,0.962333,2025,130,77.24
1,2025,5,258,Virginia,Florida State,52,46,38,2025,1576,...,2025,0.262982,0.116078,0.472059,0.373303,1.047901,0.948577,2025,19,244.11
2,2025,5,9,Arizona State,TCU,2628,27,24,2025,1724,...,2025,0.236800,0.168415,0.449150,0.401299,0.971079,0.970145,2025,24,229.80
3,2025,5,204,Oregon State,Houston,248,24,27,2025,1298,...,2025,0.151041,0.146870,0.383250,0.375206,1.005758,0.988781,2025,52,198.73
4,2025,5,356,Illinois,USC,30,34,32,2025,1578,...,2025,0.373025,0.116049,0.531181,0.400356,1.007541,0.951917,2025,14,257.02


In [6]:
predictions.head()

,season,week,homeTeam,awayTeam,pred_homePoints,pred_awayPoints,pred_pointDiff
0,2025,5,East Carolina,Army,30.55,29.65,0.90
1,2025,5,Virginia,Florida State,30.54,28.94,1.60
2,2025,5,Arizona State,TCU,30.86,26.93,3.93
3,2025,5,Oregon State,Houston,19.85,29.25,-9.40
4,2025,5,Illinois,USC,24.73,32.31,-7.58


In [7]:
merged = pd.merge(
    actual,
    predictions,
    on=['season', 'week', 'homeTeam', 'awayTeam'],
    suffixes=('_actual', '_pred')
)


In [8]:
merged.columns	

Index(['season', 'week', 'homeId', 'homeTeam', 'awayTeam', 'awayId',
       'homePoints', 'awayPoints', 'home_year_x', 'home_elo', 'home_fpi',
       'home_rating', 'home_offense', 'home_defense', 'home_year_y',
       'home_epa', 'home_epaAllowed', 'home_successRate',
       'home_successRateAllowed', 'home_explosiveness',
       'home_explosivenessAllowed', 'home_year', 'home_rank', 'home_points',
       'away_year_x', 'away_elo', 'away_fpi', 'away_rating', 'away_offense',
       'away_defense', 'away_year_y', 'away_epa', 'away_epaAllowed',
       'away_successRate', 'away_successRateAllowed', 'away_explosiveness',
       'away_explosivenessAllowed', 'away_year', 'away_rank', 'away_points',
       'pred_homePoints', 'pred_awayPoints', 'pred_pointDiff'],
      dtype='object')

In [ ]:
# Merge on matchup columns (adjust if your column names differ)
merged = pd.merge(
    actual,
    predictions,
    on=['season', 'week', 'homeTeam', 'awayTeam'],
    suffixes=('_actual', '_pred')
)

# Calculate absolute error for home and away scores
merged['home_error'] = (merged['homePoints'] - merged['pred_homePoints']).abs()
merged['away_error'] = (merged['awayPoints'] - merged['pred_awayPoints']).abs()

# Calculate mean absolute error
mae_home = merged['home_error'].mean()
mae_away = merged['away_error'].mean()

# Determine if the predicted winner matches the actual winner
merged['actual_winner'] = merged.apply(
    lambda row: 'home' if row['homePoints'] > row['awayPoints']
    else ('away' if row['homePoints'] < row['awayPoints'] else 'tie'),
    axis=1
)
merged['pred_winner'] = merged.apply(
    lambda row: 'home' if row['pred_homePoints'] > row['pred_awayPoints']
    else ('away' if row['pred_homePoints'] < row['pred_awayPoints'] else 'tie'),
    axis=1
)
merged['correct_prediction'] = merged['actual_winner'] == merged['pred_winner']

# Calculate accuracy
accuracy = merged['correct_prediction'].mean()

print(f"Mean Absolute Error (Home): {mae_home:.2f}")
print(f"Mean Absolute Error (Away): {mae_away:.2f}")
print(f"Winner Prediction Accuracy: {accuracy:.2%}")

merged[['season', 'week', 'homeTeam', 'awayTeam', 'homePoints', 'awayPoints', 'pred_homePoints', 'pred_awayPoints', 'actual_winner', 'pred_winner', 'correct_prediction']].head()

Mean Absolute Error (Home): 10.01
Mean Absolute Error (Away): 9.06
Winner Prediction Accuracy: 68.63%


,season,week,homeTeam,awayTeam,homePoints,awayPoints,pred_homePoints,pred_awayPoints,actual_winner,pred_winner,correct_prediction
0,2025,5,East Carolina,Army,28,6,30.55,29.65,home,home,True
1,2025,5,Virginia,Florida State,46,38,30.54,28.94,home,home,True
2,2025,5,Arizona State,TCU,27,24,30.86,26.93,home,home,True
3,2025,5,Oregon State,Houston,24,27,19.85,29.25,away,away,True
4,2025,5,Illinois,USC,34,32,24.73,32.31,home,away,False
